# Superstore Sales Dashboard — Data Analysis
**Author:** Nick Trybushkin  
**Goal:** Perform exploratory data analysis and build a Power BI dashboard using Superstore dataset.  
**Tools:** Python (Pandas, NumPy, Matplotlib, Seaborn)


In [ ]:
import pandas as pd

file_path = '../data/Sample - Superstore.csv'

df = pd.read_csv(file_path, encoding='ISO-8859-1')

# df.head()
# df.shape
# df.info()

df.columns = [col.strip().replace(" ","_").lower() for col in df.columns]
# df.sample(100).to_csv("../data/superstore_sample.csv", index=False)

# C. Identify numeric and non-numeric fields

# df.select_dtypes(include="number").columns
# df.select_dtypes(exclude="number").columns

# D. Check for missing or null values
# df.isna().sum().sort_values(ascending=False)

# E. Basic statistics summary
df.describe()

# F. Inspect column names manually
# df.columns.tolist()


### Initial observations

- Dataset has 9994 rows with 21 columns.
- No critical missing values found.
- 'order_date' and 'ship_date' will need to be converted to datetime.
- Sales and profit are highly skewed distributions with several extreme outliers - particulary large Sales values and both high and low Profit values.
- Discount ranges from 0 - 0.8, which may influance profitability.
- Region, Category, and Sub-category are categorical and suit grouping.

In [ ]:
# 4. Basic Statistics Check
# Goal:

# Use descriptive statistics and simple Pandas checks to detect numerical issues such as outliers, extreme discounts, negative profits, or inconsistent values.

# Generate Descriptive Statistics
df.describe().transpose()

# Inspect key numeric columns individually

df['sales'].describe()
# has extremely high max value: 22638.480000
# because of that mean is high: 229 and std: 623 is larger than mean

df['profit'].describe()
# extremely min: -6599 and max: 8399
# std: 234 is larger than mean: 28

df['discount'].unique()[:10]
# no values > 1

df['discount'].describe()
# std: 0.2 > mean: 0.15

df['quantity'].describe()
# no 0 or extra large values - no issues

# C. Detect outliers with simple conditions
df[df['discount'] > 0.8]
# usually discout shouldn't be higher than 0.8, if there are such , need follow-up
# no such rows

df[df['profit'] < -500]
# 50 rows, will keep them
# will heavily influence visualization

df[df['quantity'] % 1 != 0]
# shoudln't be any decimal numbers

df[df['sales'] > 5000]
# 19 such rows
# rare but legitimate

# Check numeric relationships

df[df['profit'] > df['sales']]
# 0 rows
# profit shouldn't be more than sales
# if not, investigate


df[['quantity', 'sales']].corr()
# expect to see the moderate positive correlation, here: 0.2
# if no, require investigation

# Profit should not be 0 when sales > 0
df[(df['sales'] > 0) & (df['profit'] == 0)]
# Found 65 rows, should be rare

# Discount should reduce profit
df[(df['discount'] > 0.5) & (df['profit'] > 0)]
# found 0 rows, as high discound and profit are suspicious


### Basic Statistic Observation

- Sales range from 0.44 to 22638, because of that mean is high: 229 and std: 623 is larger than mean.
- Profit range from -6599 to 8399, negative values occur because of the large discount, std: 234 is much larger than mean: 28.
- Discounts vary from 0 to 0.8, no values above 0.8 detected, std: 0.2 > mean: 0.15.
- Quantity has range from 1 to 14; all values appear reasonable.
- No inconsinstancies found (e.g. profit > sales).
- Outliers will be kept for now, but need special attention during visualization.


In [ ]:
# Quick Structure Understanding

# List all categorical columns
categorical_cols = df.select_dtypes(exclude='number').columns
df[categorical_cols].nunique().sort_values(ascending=False)

# Very high-cardinality columns (like product_name, customer_name) → useful for top/bottom analysis.
# Medium-cardinality columns: city, state
# Low-cardinality columns: region, category, segment, ship_mode → great for grouping.

# Inspect key categorical fields

# Category
df['category'].value_counts()
# expect: Office Supplies > Furniture > Technology, it's correct

# sub-category
df['sub-category'].value_counts().head()
# shows detailed product types

# region
df['region'].value_counts()

# ship_mode
df['ship_mode'].value_counts()

# customer segment
df['segment'].value_counts()

# These are dimensions i will analyze mostly

# Look at relationships between key categories
df.groupby('category')['sub-category'].nunique()

df.groupby('region')['state'].nunique()

# Quick peek
df.groupby('category')['profit'].sum().sort_values(ascending=False)

# Identify high-level dimensions
# Main buisness dimension to analyze

# Time -> order_date, ship_date
# Location -> region, state, city
# Customer -> segment, customer_name
# Product -> category, sub-category, product_name
# Shipping -> ship_mode
# Metrics -> sales, profit, discount, quantity




### Structure Understanding — Key Insights

- Dataset contains several important categorical dimensions:
  - **Products:** Category (3), Sub-category (17), Product Name (~1850)
  - **Geography:** Region (4), State (~49), City (~500)
  - **Customers:** Segment (3), Customer Name (~800)
  - **Shipping:** Ship Mode (4)
- Product and geography dimensions will be crucial for KPI analysis.
- High-cardinality fields (product_name, customer_name) are useful for ranking (top/bottom lists).
- Category → Sub-category relationship is well-defined and consistent.
- Region → State → City hierarchy appears logical and complete.

In [ ]:
# 6. Data Cleaning
# Goal:

# Fix data types, remove duplicates, standardize values, create new useful columns.
# This prepare the dataset for analysis and PowerBI dashboarding

# A. Convert Date columns to Datetime

# 1. Identify date columns
# order_date, ship_date

# 2. Convert
df['order_date'] = pd.to_datetime(df['order_date'])
df['ship_date'] = pd.to_datetime(df['ship_date'])

# 3. Verify
df[['order_date', 'ship_date']].dtypes

# 4. Check for conersion errors
df[df['order_date'].isna()]
df[df['ship_date'].isna()]

# 5. Optional, extract components (will be used later)
df['order_year'] = df['order_date'].dt.year
df['order_month'] = df['order_date'].dt.month
df['order_day'] = df['order_date'].dt.day

df.head()

# B. Sort Dataset by Order Date
# Chronologically organize the data so all time-based analysis (trends, growth, seasonality) is correct and intuitive

# 1. Sort values
df = df.sort_values('order_date')

# 2. Reset index
# drop=True ensures Pandas won't add the old index as column
df = df.reset_index(drop=True)

# 3. Verify sorting work
# print(df[['order_date', 'ship_date']].head())
# print(df[['order_date', 'ship_date']].tail())

# 4. Optionally, sort both order and ship dates
df = df.sort_values(['order_date', 'ship_date'])

# 5. Validate chronological correctness
# shipping should always occur after order
df[df['ship_date'] < df['order_date']]
# should be empty

# Summary
# We have
# Proper datetime columns
# Chronologically ordered data
# Clean index
# Validated date structure 

# C. Check and remove duplicates
# Identify and remove duplicate rows to avoid inflating metrics like total sales, total profit, etc.

# 1. Check duplicates
df.duplicated().sum()

# 2. Preview duplicate rows (if any)
df[df.duplicated()].head()

# 3. Drop duplicates
df = df.drop_duplicates()
df = df.reset_index(drop=True)
# print(df.head())

# 4. Check for duplicated order IDs
# order id can repeat , as single order can have multiple products

df['order_id'].nunique(), df.shape[0]
# Interpretation:
# fewer unique order IDs than rows → correct
# many items per order expected

# Summary, you only remove the rows where every column identical

# D. Understand if any column has missing values and decide what to do with them.

# 1. Count missing values
df.isna().sum().sort_values(ascending=False)

# 2. Visual scan of missing data, sometimes useful
# import seaborn as sns
# import matplotlib.pyplot as plt

# sns.heatmap(df.isna(), cbar=False)
# plt.show()

# This gives a matrix-like visualization.
# If you see bright vertical lines → many missing values.
# Normally, Superstore dataset will look mostly solid (few or no gaps).


# If missing values exist - choose strategy

# 1. For categorical column

# Examples: region, segment, ship_mode

# Options:

# Fill with most frequent value
# df['column'] = df['column'].fillna(df['column'].mode()[0])

# Fill with unknown
# df['column'] = df['column'].fillna('unknown')

# For numeric columns
# exmaple: sales, profit, discount

# options

# Fill with 0
# df['column'] = df['column'].fillna(0)

# Fill with median
# df['column'] = df['column'].fillna(df['column'].median())

# For Date columns
# Drop only if all missed

# df = df.dropna(subset=['order_date','ship_date'])

# Verify no missing values remain
df.isna().sum().sum()

# Check for empty strings
(df == "").sum()
# If any column shows count > 0, treat it like missing values 

# Summary of Step D

# By the end of this step, you will:
# Know exactly which columns had missing values
# Apply appropriate imputation if needed
# Confirm the cleaned dataset has no NaNs or empty fields that break analysis

# E. Validate Numeric Consistency

# Goal: check that numeric fields follow business rules
# This protects your analysis from bad data (e.g. negative quantities, impossible discounts)

# 1. Check for invalid quintity value

# Quantity should always be 1 or more
df[df['quantity'] < 1]

# Fix if needed
df = df[df['quantity'] > 0]

# 2. Validate discount values
# Discount must be > 0 and < 1
df[(df['discount'] < 0) | (df['discount'] > 1)]

# Fix if needed
df.loc[df['discount'] > 1, 'discount'] = 1
df.loc[df['discount'] < 0, 'discount'] = 0

# Check that profit is not greater than sales
df[df['profit'] > df['sales']]

# Should be empty, if not, can be due to incorrect discount
# Example investigation
df[df['profit'] > df['sales']].head()
# than decide what to do (drop or adjust)

# 3. Look for suspicious negative sales
# sales should never be negative
df[df['sales'] < 0]

# Fix if neede
df = df[df['sales'] >= 0]

# profit can be, because of high discount
# sales - never

# 4. Check for zero or near-zero sales
df[df['sales'] < 1].head()

# sometimes tiny values are normal
# but if it's 0.00 may indicate incorrect entries.

### Numeric consistency check

- No negative quantities found.
- Discounts range is from 0 to 0.8.
- No cases when profit more than sales.
- No negative sales.

In [ ]:
# F. Detect outliers without removing them

# Identify unusual alues that can distort graphs or analysis
# We do NOT remove outliers, they are often real business cases, especially discounts

# 1. Detect high discounts
df[df['discount'] > 0.5].head()

# Deep discounts often produce negative profits

# 2. Detect extremely negative profits
# Profit below -2000 is rare and usually indicates the large discount item

df[df['profit'] < -2000].head()

# These values are valid but important for
# Boxplot
# Distibutions
# Correlation analysis
# You will later mention them as outliers later

# 3. Detect unusual high sales values
df[df['sales'] > 5000].head()
# These are usually legitimate large furniture purchases

# 4. Look at distribution shapes

# Sales distribution
# df['sales'].plot(kind='hist', bins=50, figsize=(7,4))

# Profit distribution
# df['profit'].plot(kind='hist', bins=50, figsize=(7,4))

# You will notice: 
# Profit has many negative values
# Sales is right skewed, typically in retail

# 5. Optional: use IQR method to flag outliers, NOT DELETE
q1 = df['sales'].quantile(0.25)
q3 = df['sales'].quantile(0.75)
iqr = q3 - q1

outliers = df[(df['sales'] < q1-iqr*1.5) | (df['sales'] > q3+iqr*1.5)]
outliers.head()

### Outliers analysis

- High discounts (> 50%) found primarily in Furniture category.
- Several extremely negative profits (<-2000) observed - all result of high discounts.
- Large sales values (> 5000) represent big furniture orders.
- Profit distribution is heavily tailed with many negative cases.
- Outliers will be kept for analysis, but noted in reporting.


In [7]:
# G. Create new calculated columns
# Add derived variables that enhance your analysis, and help answer business questions.
# These fields will be used again and again in grouping, filtering and visualizing results

# 1. Profit Margin
# Shows how much of each dollar of each sale is profit

df['profit_margin'] = df['profit'] / df['sales']
# Can be negative (loss)
# Can't be > 1 (if so, investigate later)

df['profit_margin'].describe()

# Key insights:

# The mean profit margin is 12%, which indicates that the business operates with relatively low overall margins.
# The median margin is significantly higher (27%), suggesting that most transactions are profitable and fall into the 20–30% range.
# The minimum margin is –275%, indicating the presence of extreme loss-making orders, likely caused by heavy discounts, returns, or data anomalies.
# The maximum margin is 50%, which is realistic for certain high-markup items.
# The standard deviation is very high (0.466), reflecting large variability in profitability across products and categories.
# The data includes notable negative outliers, which pull the mean down and should be investigated separately.

# Business Implication:

# The company is mostly profitable, but a small number of orders — likely related to discounts, logistics costs, or data 
# quality issues — generate disproportionately large losses. These outliers need further investigation to determine whether they represent valid business scenarios or data inconsistencies.

# 2. Order year
df['order_year'] = df['order_date'].dt.year
# Grouping by year is essential for YOY (year-over-year) analysis

df['order_year'].value_counts().sort_values(ascending=False)

# 3. Order month
# For monthly trend analysis
df['order_month'] = df['order_date'].dt.month
df['order_month'].value_counts().sort_values()

# 4. Month-year combination
df['order_month_year'] = df['order_date'].dt.to_period('M').astype(str)
df['order_month_year'].value_counts().sort_values()
# Very useful for continues trend plots (e.g. line chart)

# 5. Shipping delay (days)
# Difference between ship_date and order_date indicates delivery speed
df['shipping_delay'] = (df['ship_date'] - df['order_date']).dt.days
df['shipping_delay'].describe()

# Flag negative delays, should be none
df[df['shipping_delay'] < 0]

# 6. Revenue buckets (categorical-bins)
# Useful for segmentation in dashboards
df['sales_bucket'] = pd.cut(df['sales'], bins=[0,100,500,1000,5000,20000], labels=['0-100','100-500','500-1000','1000-5000','5000+'])
df['sales_bucket'].value_counts().sort_values()

# H. Verify calculated columns
# Insure new created columns correct

# 1. Preview all new columns
df[['sales', 'profit', 'profit_margin', 'order_year', 'order_month', 'order_month_year', 'shipping_delay']].head()
# check no NaN values
# month and year appear correct
# month_year in YYYY-MM format
# shipping delay positive

# 2. Validate profit margin
df[df['profit_margin'] > 1]

# check for extreme negatives
df[df['profit_margin'] < -1]
# 349 rows, due to strong discounts

# 3. Validate shipping delay
df['shipping_delay'].describe()

df[df['shipping_delay'] > 20]

# 4. Validate month-year continuity
df['order_month_year'].nunique()

# Expected ~48 months

# 5. Validate year and month distributions
df['order_year'].value_counts()
df['order_month'].value_counts().sort_index()
# These should show regular patters, not anomalies

# 6. Check for NaN in new columns
df[['sales', 'profit', 'profit_margin', 'order_year', 'order_month', 'order_month_year', 'shipping_delay']].isna().sum()

# Summary: ready for EDA (Exploratory Data Analysis)

# I: Save cleaned dataset

# Save cleaned and enriched dataset for future EDA and PowerBI

# 1. Save as CSV
df.to_csv('../data/superstore_clean.csv', index=False)
# index have no business meaning and will pollute dataset

# 2. Save as Excel
df.to_excel('../data/superstore_clean.xlsx', index=False)

# 3. Verify files
# CSV
df_test = pd.read_csv("../data/superstore_clean.csv")
df_test.head()

# Excel
df_test = pd.read_excel("../data/superstore_clean.xlsx")
df_test.head()

# Make sure all columns present
# Datatypes look correct
# No duplicate header row
# Row count matches your cleaned dataset

# 4. Save a lightweight sample for PowerBI testing
df.sample(200).to_csv('../data/superstore_clean_sample.csv', index=False)
# This speeds up dashboarding prototyping

### Data Cleaning Summary

### Date Columns

- Converted 'order_date' and 'ship_date' to datetie format.
- Sorted entire dataset by 'order_date' -> 'ship_date'.
- Varified there are no invalid date values.

### Duplicated and Missing Values

- Found and removed **0** duplicate rows.
- Check for missing values - no critical NaNs identified.

### Numeric Consistency Checks

- Quantity: all values positive.
- Discount: all values from 0 to 0.8.
- No values where profit exceeded sales.
- All shipping dates occur after or the same date as order date.

### Outliers Analysis

- High discounts (> 50%) found primarily in Furniture category.
- Several extremely negative profits (< -2000) observed - all result of high discounts.
- Large sales values (> 5000) represent big furniture orders.
- Profit distribution is heavily tailed with many negative cases.
- Outliers will be kept for analysis, but noted in reporting.

### New Features Created

- 'profit_margin' = profit / sales
- 'order_year' (2014-2017)
- 'order_month' (1-12)
- 'month_year' (Period, useful for monthly trends)
- 'shipping_delay' in days

### Output Files Saved
- 'superstore_clean.csv' - cleaned dataset for EDA and BI
- 'superstore_clean.xlsx' - cleaned excel version for PowerBI
- Notebook updated with all cleaning steps





In [ ]:
# 1. Sales & Profit Overview

# 1.1 High-Level KPI Summary

# Goal: Get Basic financial metrics of the dataset

total_sales = df['sales'].sum()
total_profit = df['profit'].sum()
total_quantity = df['quantity'].sum()
avg_profit_margin = df['profit_margin'].mean()

# print(f"total_sales: {total_sales}")
# print(f"total_profit: {total_profit}")
# print(f"total_quantity: {total_quantity}")
# print(f"avg_profit_margin: {avg_profit_margin}")

# Interpretation:
# Total Sales: the total revenue
# Total Profit: actual earnings after costs
# Total Quantity: number of units sold
# Average Profit Margin: overall business profitability

# 1.2 Sales by Category
# Goal: see which categories generate the most revenue

sales_by_category = df.groupby('category')['sales'].sum().sort_values(ascending=False) 
# sales_by_category

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,5))
# sns.barplot(
#     x=sales_by_category.index,
#     y=sales_by_category.values
# )
# plt.title("Sales by Category")
# plt.xlabel("Category")
# plt.ylabel("Total Sales")
# plt.show()


# Interpretation:
# Technology has the highest revenue.
# Furniture is second
# Office Suppliers is third

# Key question to answer: which categories deserve more investments, based on revenues

# 1.3 Profit By Category 
# Profit tells a different story from Sales

profit_by_category =  df.groupby('category')['profit'].sum().sort_values(ascending=False) 
profit_by_category

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,5))
# sns.barplot(
#     x=profit_by_category.index,
#     y=profit_by_category.values
# )
# plt.title("Profit by Category")
# plt.xlabel("Category")
# plt.ylabel("Total Profit")
# plt.show()


# Interpretation:
# Technology has the highest profit.
# Office Suppliers is second
# Furniture is third, with VERY LOW PROFIT

# Question to Answer: are we selling products that bring revenue but don't bring profit


# 1.4 Profit Margin by Category

# Goal
# Show how efficient every category is
avg_profit_margin_by_category =  df.groupby('category')['profit_margin'].mean().sort_values(ascending=False) 
avg_profit_margin_by_category

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,5))
# sns.barplot(
#     x=avg_profit_margin_by_category.index,
#     y=avg_profit_margin_by_category.values
# )
# plt.title("Avg Profit Margin by Category")
# plt.xlabel("Category")
# plt.ylabel("Avg Profit Margin")
# plt.show()

# Interpretation:
# Categories with high sales but low margin is risky (Furniture)
# Categories with low sales but high margin can be opportunity

# 1.5 Sales and profit by Sub-Category

# Goal
# Deepen the analysis: which product types drive results

sub_metrics = (
    df.groupby('sub-category')[['sales', 'profit']]
      .sum()
      .sort_values('profit', ascending=False)   # optional sorting
)

# import matplotlib.pyplot as plt

# sub_metrics.plot(kind='bar', figsize=(12,6))
# plt.title('Sales & Profit by Sub-Category')
# plt.xlabel('Sub-Category')
# plt.ylabel('Amount')
# plt.xticks(rotation=45, ha='right')
# plt.tight_layout()
# plt.show()

# Interpretation:
# High sales but low profit: Phones, Chairs, Storage
# Loss making sub-categories: Tables, Bookcases, Supplies
# Hidden gems with high margin: Paper, Copiers, Accessories, Envelopes, Labels

# 1.6 Sales & Profit By Segment
# Identify, which custommer segment is most valuable

sub_metrics = (
    df.groupby('segment')[['sales', 'profit']]
      .sum()
      .sort_values('profit', ascending=False)   # optional sorting
)

# import matplotlib.pyplot as plt

# sub_metrics.plot(kind='bar', figsize=(12,6))
# plt.title('Sales & Profit by Segment')
# plt.xlabel('Segment')
# plt.ylabel('Amount')
# plt.xticks(rotation=45, ha='right')
# plt.tight_layout()
# plt.show()

# Interpretation
# Consumer has the highest revenue.
# Corporate also has strong profit.
# Home Office is the smallest but highly profitable.

# Key business questions:
# Which segment is growing or shrinking?
# Should marketing focus on Consumer or Corporate clients?

# 1.7. Combined Sales & Profit into a Single Overview Table

overview = df.groupby('category').agg({
    'sales': 'sum',
    'profit': 'sum',
    'profit_margin': 'mean'
}).sort_values(by='profit', ascending=False)

print(overview)

overview = df.groupby('segment').agg({
    'sales': 'sum',
    'profit': 'sum',
    'profit_margin': 'mean'
}).sort_values(by='profit', ascending=False)

print(overview)

# Will be added to README



- **Technology** leads in both sales and profit, indicating strong demand and good margins.

- **Furniture** generates high revenue but weak profit, suggesting pricing or cost issues.

- Several sub-categories (e.g., **Tables**, **Bookcases**, **Supplies**) show negative profit despite strong sales.

- The **Consumer** segment is the largest revenue driver, but Corporate appears more efficient.
**Home Office** has the lowest Sales but the biggest profit margin 


In [ ]:
# 2. Product Performance Analysis
# In this section we will analyze individual performance to identify

# - Best selling product
# - Most profitable product
# - Products that cause financial loss
# - Profit margin distribution across products

# 2.1. Top 10 Products by Sales
# Goal: find products generating the highest revenue.

top_sales = (
    df.groupby(['category', 'product_name'])['sales']
      .sum()
      .sort_values(ascending=False)
      .head(10)
)
# print(top_sales)

# Prepare data for plot
top_sales_plot = top_sales.copy()
top_sales_plot.index = top_sales_plot.index.map(lambda x: f"{x[0]} — {x[1]}")

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,6))
# sns.barplot(
#     x=top_sales_plot.values,
#     y=top_sales_plot.index
# )
# plt.title("Top 10 Products by Sales")
# plt.xlabel("Total Sales")
# plt.ylabel("Product Category + Name")
# plt.show()

# Interpretation:
# Technologies and Office Suppliers dominate revenue.
# Leads 'Technology - Canon imageCLASS 2200 Advanced Copier' with a big gap  

# 2.2. Top 10 Products by Profit
# Goal: identify items that actually generate earnings.

top_profit = (
    df.groupby(['category','product_name'])['profit']
      .sum()
      .sort_values(ascending=False)
      .head(10)
)

# Prepare data for plot
top_profit_plot = top_profit.copy()
top_profit_plot.index = top_profit_plot.index.map(lambda x: f"{x[0]} — {x[1]}")

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,6))
# sns.barplot(
#     x=top_profit_plot.values,
#     y=top_profit_plot.index
# )
# plt.title("Top 10 Products by Profit")
# plt.xlabel("Total Profit")
# plt.ylabel("Product Category + Name")
# plt.show()

# Interpretation:
# Technology category dominate profit.
# Office supliers follow with big gap.
# Leads 'Technology - Canon imageCLASS 2200 Advanced Copier' with a big gap  

# 2.3. Bottom 10 Products by Profit (Worst Performers)
# Goal: reveal products that consistently lose money.

bottom_profit = (
    df.groupby(['category', 'product_name'])['profit']
      .sum()
      .sort_values()
      .head(10)
)

# Prepare data for plot
bottom_profit_plot = bottom_profit.copy()
bottom_profit_plot.index = bottom_profit_plot.index.map(lambda x: f"{x[0]} — {x[1]}")

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,6))
# sns.barplot(
#     x=bottom_profit_plot.values,
#     y=bottom_profit_plot.index
# )
# plt.title("Bottom 10 Products by Profit (Most Unprofitable)")
# plt.xlabel("Total Profit")
# plt.ylabel("Product Category + Name")
# plt.show()

# Interpretation:

# Most losses come from high-cost Technology products (3D printers, laser printers, conferencing systems).
# Several bulky Furniture items (conference tables) are also strongly unprofitable.
# A few Office Supplies products appear, indicating potential pricing or overstock issues.
# Top three items show very large negative profits, far worse than others.

# Key question:
# Why are we selling products that consistently produce losses?

# 2.4. Profit Margin Distribution Across Products
# Goal: understand how profitable individual products are on average.

pm_distribution = (
    df.groupby(['category','product_name'])['profit_margin']
      .mean()
)

pm_distribution.describe()

# Visualization: Histogram
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(8,5))
# sns.histplot(pm_distribution, bins=30, kde=True)
# plt.title("Distribution of Average Profit Margin Across Products")
# plt.xlabel("Average Profit Margin")
# plt.ylabel("Count of Products")
# plt.show()

# Interpretation:

# Most products have positive profit margins, clustered between 0% and 40%.
# A noticeable group of products has low or near-zero margins, indicating weak profitability.
# A small but important tail of products shows strongly negative margins, meaning they lose money.
# The distribution is right-skewed, with many moderately profitable items and few extreme losses.
# Negative-margin products should be reviewed for pricing, discounts, or high return rates.


# 2.5. Identify “High Sales but Low Profit” Products

# Goal: find items that sell well but don’t bring profit — a major business risk.

product_stats = df.groupby(['category', 'product_name']).agg({
    'sales': 'sum',
    'profit': 'sum',
    'profit_margin': 'mean'
})

product_stats.index = product_stats.index.map(lambda x: f"{x[0]} — {x[1]}")

high_sales_low_profit = (
    product_stats[(product_stats['sales'] > product_stats['sales'].median()) & 
                  (product_stats['profit'] < 0)]
    .sort_values(by='sales', ascending=False)
)

high_sales_low_profit.head(10)

# Interpretation

# These products:
# Generate strong revenue
# But damage total profit
# Usually tied to large discounts or high shipping cost items
# Business action: review pricing or discontinue.


# 2.6. Product-Level Correlation Check
# Goal: check if sales and profit correlate at product level.

product_corr = product_stats[['sales', 'profit']].corr()
# print(product_corr)

# Interpretation

# The correlation (0.59) is moderately positive but not perfect.
# This confirms that high sales ≠ high profit.

# 2.7. Summary Table of Key Product Metrics

# This table is perfect for future dashboard/report use.

product_overview = product_stats.sort_values(by='profit', ascending=False).head(15)
product_overview


### Product Performance Analysis
- Technologies and Office Suppliers dominate revenue.
- Technology category dominate profit & Office supliers follow with big gap.
- Most losses come from high-cost Technology products (3D printers, laser printers, conferencing systems). Several bulky Furniture items (conference tables) are also strongly unprofitable.
- Most products have positive profit margins, clustered between 0% and 40%.  A noticeable group of products has low or near-zero margins, indicating weak profitability.
- The correlation (0.59) between sales and profit is moderately positive but not perfect. This confirms that high sales ≠ high profit.



In [ ]:
# 3. Customer Performance Analysis

# Customers drive revenue and profit.In this section we analyze:
# Top customers by sales
# Top customers by profit
# Most unprofitable customers
# Customer segmentation (high/middle/low value)
# Profit margin by Segment

# Goal: understand which customers contribute most to business and which customers may cause losses due to returns, high discounts, or heavy shipping costs.

# 3.1 Top 10 customers by sales

top_customers_sales = df.groupby(['segment', 'customer_name'])['sales'].sum().sort_values(ascending=False).head(10)
top_customers_sales.index = top_customers_sales.index.map(lambda x: f"{x[0]} — {x[1]}")

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,6))
# sns.barplot(
#     x=top_customers_sales.values,
#     y=top_customers_sales.index
# )
# plt.title("Top 10 Customers by Sales")
# plt.xlabel("Total Sales")
# plt.ylabel("Customer Segment + Name")
# plt.show()

# Interpretation
# The top customer by sales is Sean Miller (Home Office), far ahead of all others.
# Corporate segment customers appear less frequently but generate very high order values, represented by Tamara Chand.
# Most top customers belong to the Consumer segment, but their spending is more evenly distributed.

# 3.2 Top 10 customers by profit

top_customers_profit = df.groupby(['segment', 'customer_name'])['profit'].sum().sort_values(ascending=False).head(10)
top_customers_profit.index = top_customers_profit.index.map(lambda x: f"{x[0]} — {x[1]}")

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,6))
# sns.barplot(
#     x=top_customers_profit.values,
#     y=top_customers_profit.index
# )
# plt.title("Top 10 Customers by Profit")
# plt.xlabel("Total Profit")
# plt.ylabel("Customer Segment + Name")
# plt.show()

# Interpretation
# Corporate customer, Tamara Chand, is the clear top contributor.
# The top customer by sales, Sean Miller (Home Office), is NOT in the list 
# Most other top-profit customers come from the Consumer segment, with strong but more evenly distributed profit contributions.
# Home Office customers appear less frequently but still include solid profit generators.
# High-profit customers are spread across all segments, indicating broad profitability rather than reliance on a single segment.

# 3.3 Bottom 10 customers by profit (Loss-Makers)

bottom_customers_profit = df.groupby(['segment', 'customer_name'])['profit'].sum().sort_values().head(10)
bottom_customers_profit.index = bottom_customers_profit.index.map(lambda x: f"{x[0]} — {x[1]}")

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,6))
# sns.barplot(
#     x=bottom_customers_profit.values,
#     y=bottom_customers_profit.index
# )
# plt.title("Bottom 10 Customers by Profit (Highest Losses)")
# plt.xlabel("Total Profit")
# plt.ylabel("Customer Segment + Name")
# plt.show()

# Interpretation
# The largest loss-maker is Cindy Stewart (Consumer) with a significantly higher negative impact than all others.
# Loss-making customers appear across all segments (Consumer, Corporate, Home Office), showing it’s not isolated to one segment. BUT
# Mostly Corporate and a few Home Office customers contribute substantial losses, suggesting pricing, discounting, or shipping cost issues for certain orders.

# 3.4. Customer Segmentation (High / Mid / Low Value)

# Goal: segment customers by total profit or revenue
# Classic segmentation = Pareto 20/60/20 rule.

customers_profit = df.groupby('customer_name')['profit'].sum().sort_values(ascending=False)

import numpy as np

high_treshold = np.percentile(customers_profit, 80)
low_treshold = np.percentile(customers_profit, 20)

def segment_customer(p):
    if p >= high_treshold:
        return 'High Value'
    elif p <= low_treshold:
        return 'Low Value'
    else:
        return 'Mid Value'
    
customers_segmentation = customers_profit.apply(segment_customer)
customers_segmentation.value_counts()

# Interpretation
# Mid-Value customers are the largest group (475 customers), representing the core of the customer base. They generate steady but moderate profit and form the bulk of recurring business.

# High-Value customers (159 customers) are a much smaller group but likely contribute a disproportionately large share of total profit. These are the most profitable and strategically important customers.

# Low-Value customers (159 customers) form the smallest—but important—segment. Many of them produce low or negative profit and may require pricing, discount, or service-cost optimization.

# 3.5 Merge Segmentation Back to the Data
# Analyze average behaviour per segment

df_customers = df.copy()
df_customers['customer_segment'] = df_customers['customer_name'].map(customers_segmentation)


# 3.6 Profit Margin by Customer Segment
# Goal: understand which customer segment is most efficient

customer_segment_pm = df_customers.groupby('customer_segment')['profit_margin'].mean().sort_values(ascending=False)
# print(customer_segment_pm)

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(7,4))
# sns.barplot(
#     x=customer_segment_pm.index,
#     y=customer_segment_pm.values
# )
# plt.title("Average Profit Margin by Customer Segment")
# plt.xlabel("Customer Segment")
# plt.ylabel("Avg Profit Margin")
# plt.show()

# Interpretation

# High Value customers generate high margins (0.188).
# Low Value customers may generate negative margins (-0.028).
# Mid Value customers show stable but average profitability (0.136).

# 3.7 Summary Table of Segment-Level Metrics

customer_segment_metrics = df_customers.groupby('customer_segment').agg({
    'sales': 'sum',
    'profit': 'sum',
    'profit_margin': 'mean',
    'customer_name': 'nunique'
}).rename(columns={'customer_name': 'unique_customers'})
customer_segment_metrics

# High-Value customers (159 customers) generate the largest total profit (233.885 $) and the highest profit margin (18.8%), making them the most valuable segment for the business.

# Mid-Value customers (475 customers) form the majority of the customer base and contribute strong (123.734 $), stable revenue with healthy profit margins (13.6%).

# Low-Value customers (159 customers) generate substantial revenue but produce negative total profit (-71.222 $), indicating heavy discounts, high costs, or unprofitable buying behavior.

# Overall, profitability is concentrated in the High-Value and Mid-Value groups, while the Low-Value segment represents a clear opportunity for pricing or cost optimization.


### Customer Performance Analysis

- Most top customers belong to the Consumer segment, but their spending is more evenly distributed.
- Most top-profit customers come from the Consumer segment, with strong but more evenly distributed profit contributions. Home Office customers appear less frequently but still include solid profit generators.
- Loss-making customers appear across all segments (Consumer, Corporate, Home Office), showing it’s not isolated to one segment. BUT Mostly Corporate and a few Home Office customers contribute substantial losses.
- Overall, profitability is concentrated in the High-Value and Mid-Value groups, while the Low-Value segment represents a clear opportunity for pricing or cost optimization.

In [11]:
# 4. Time Series Trends

# Understanding how sales and profit evolve over time helps identify seasonality, growth patterns, and anomalies that can guide business decisions.
# In this part of EDA, we analyze:

# Yearly trends
# Monthly trends
# Month-Year timeline
# Sales vs Profit comparison over time
# Seasonal peaks or unusual dips
# This analysis also prepares the foundation for dashboard visualizations and YoY insights.

# 4.1 Prepare Time-Based Columns (if not already done)
# Goal: ensure the dataset contains time-specific features needed for analysis.

# if already done before, just confirm
# df['order_year'] = df['Order Date'].dt.year
# df['order_month'] = df['Order Date'].dt.month
# df['month_year'] = df['order_month_year'].dt.to_period('M').astype(str)

# df[['Order Date', 'order_year', 'order_month', 'month_year']].head()

# I already have needed columns

# 4.2 Yearly Sales and Profit Trends

# Goal: get a high-level view of annual business performance.

yearly = df.groupby('order_year').agg({
    'sales': 'sum',
    'profit': 'sum'
}).reset_index()

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,5))
# sns.lineplot(data=yearly, x='order_year', y='sales', marker='o', label='Sales')
# sns.lineplot(data=yearly, x='order_year', y='profit', marker='o', label='Profit')
# plt.title("Yearly Sales & Profit Trends")
# plt.xlabel("Year")
# plt.ylabel("Amount ($)")
# plt.legend()
# plt.show()

# Interpretation

# Both sales and profit show a clear upward trend from 2014 to 2017, indicating strong overall business growth.
# After a slight dip in 2015, sales accelerate sharply in 2016 and 2017, suggesting improved demand or successful sales strategies.
# Profit increases steadily every year, growing faster than sales in percentage terms, which indicates improving efficiency or margin management.

# 4.3. Monthly Sales & Profit Trend (Seasonality)
# Goal: identify seasonal patterns (strong months, slow months).

monthly = df.groupby('order_month').agg({
    'sales': 'sum',
    'profit': 'sum'
}).reset_index()

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,5))
# sns.lineplot(data=monthly, x='order_month', y='sales', marker='o', label='Sales')
# sns.lineplot(data=monthly, x='order_month', y='profit', marker='o', label='Profit')
# plt.title("Monthly Sales Trend (Seasonality)")
# plt.xlabel("Month")
# plt.ylabel("Total Sales")
# plt.xticks(range(1,13))
# plt.show()

# Interpretation

# The business shows clear seasonality, with strong peaks in March, September, November, and December, indicating high-demand periods.
# Sales dip noticeably in January, February, and April, marking recurring low-demand periods.
# Profit follows the same pattern as sales, showing consistent growth during peak months and smaller gains during slow periods.

# 4.4. Month-Year Timeline (Full Time Series)
# Goal: visualize a continuous timeline (e.g., from 2014–2017).

order_month_year_trend = (
    df.groupby('order_month_year')[['sales', 'profit']]
      .sum()
      .reset_index()
)

# order_month_year_trend.head()

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(14,6))
# sns.lineplot(data=order_month_year_trend, x='order_month_year', y='sales', label='Sales')
# sns.lineplot(data=order_month_year_trend, x='order_month_year', y='profit', label='Profit')
# plt.title("Sales & Profit Over Time (Month-Year)")
# plt.xlabel("Month-Year")
# plt.ylabel("Amount ($)")
# plt.xticks(rotation=90)
# plt.legend()
# plt.tight_layout()
# plt.show()

# Interpretation

# Both sales and profit show strong upward momentum over multiple years, with noticeably higher peaks toward late 2016 and throughout 2017.
# The chart clearly displays monthly seasonality, with recurring spikes each year—especially in Q4 (Oct–Dec) and occasional peaks in Q1/Q2.
# Profit closely follows sales patterns but with much smaller magnitude, confirming a consistent relationship between revenue and profitability.

# 4.5. Optional: Year-over-Year Growth (YoY)
# Goal: compute annual growth to highlight acceleration or decline.

# YoY Growth (%) = (Current Year - Previous Year) / Previous Year * 100

yearly['sales_yoy'] = yearly['sales'].pct_change() * 100
yearly['profit_yoy'] = yearly['profit'].pct_change() * 100

# Interpretation

# 2015: Sales dipped slightly (–2.8% YoY), but profit grew strongly (+24.4%), showing improved margins despite flat revenue.
# 2016: Both metrics accelerated, with sales up +29.5% and profit up +32.7%, indicating a major growth year with strong operational efficiency.
# 2017: Growth continued, though at a slower pace—sales grew +20.3% and profit increased +14.2%, showing stable expansion.
# Overall, YoY results show consistent improvement, with profitability growing alongside revenue, and particularly strong performance in 2016.



### Time Series Trends Analysis

- The business shows clear seasonality, with strong peaks in March, September, November, and December, indicating high-demand periods. Profit follows the same pattern as sales, showing consistent growth during peak months and smaller gains during slow periods.
- Both sales and profit show strong upward momentum over multiple years, with noticeably higher peaks toward late 2016 and throughout 2017. Profit closely follows sales patterns but with much smaller magnitude, confirming a consistent relationship between revenue and profitability.




In [ ]:
# 5. Discount Impact Analysis

# Discounts directly influence sales volume and profit margins.

# In this section, we examine:
# How discounts relate to profit
# Whether higher discounts destroy profitability
# At what discount level profit becomes negative
# How discount behavior varies by category
# This helps identify pricing and promotion issues.

# 5.1. Scatterplot: Discount vs Profit
# Goal: visualize the relationship between discount percentage and profit on each order.

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,5))
# sns.scatterplot(
#     data=df,
#     x='discount',
#     y='profit',
#     alpha=0.4
# )
# plt.title("Discount vs Profit")
# plt.xlabel("Discount")
# plt.ylabel("Profit")
# plt.show()

# Interpretation:

# Higher discounts are strongly associated with lower profit, and many high-discount transactions produce negative profit.
# The highest profits occur almost exclusively at 0%–10% discount, showing that full-price or lightly discounted sales drive profitability.
# As discount levels rise (40%–80%), profit values cluster close to zero or deep into negative territory, indicating unprofitable discounting practices.

# 5.2. Correlation Between Discount and Profit / Profit Margin
# Goal: quantify how strong the relationship is.

df[['discount', 'profit', 'profit_margin']].corr()

# Interpretation:
# Discount has a strong negative correlation with profit margin (–0.86), meaning higher discounts almost always lead to significantly lower margins.
# There is also a moderate negative correlation between discount and total profit (–0.22), showing that discounts generally reduce profit rather than driving high-profit sales.
# Profit and profit margin show a weak positive correlation (+0.22), consistent with the idea that higher margins tend to create higher profit, but many other factors affect profit too.
# Overall, the correlations confirm that aggressive discounting is harmful to profitability, both in absolute profit and percentage margins.


# 5.3. Create Discount Bins (0–10%, 10–20%, …)
# Goal: group orders into discount ranges to see profitability at each level.

bins = [0, 0.1, 0.2, 0.3, 0.4, 1]
labels = ['0-10%', '10-20%', '20-30%','30-40%','40%+']

df['discount_bin'] = pd.cut(df['discount'], bins=bins, labels=labels, include_lowest=True)
df['discount_bin'].value_counts().sort_values(ascending=False)

# Interpretation
# Most orders fall into low-discount ranges (0–10% and 10–20%), together making up the majority of all transactions.

# 5.4. Profit by Discount Bin
# Goal: understand how profit behaves at different discount levels.

profit_by_bin = df.groupby('discount_bin')['profit'].sum().sort_index()
# profit_by_bin

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,5))
# sns.barplot(
#     x=profit_by_bin.index,
#     y=profit_by_bin.values
# )
# plt.title("Total Profit by Discount Range")
# plt.xlabel("Discount Range")
# plt.ylabel("Total Profit")
# plt.show()

# 0–10% discounts generate the overwhelming majority of total profit, making low-discount sales the core profit driver of the business.
# Profit drops sharply at 10–20% discounts, though this range still remains profitable overall.
# All discount ranges above 20% produce negative total profit, with losses increasing as discounts deepen.
# The 40%+ discount range shows the largest losses, confirming that deep discounting is highly unprofitable.
# Overall, the chart clearly demonstrates that profitability declines non-linearly with higher discounts, emphasizing the need for strict discount controls.

# 5.5. Average Profit Margin by Discount Bin
# Goal: see how efficient each discount level is.

pm_by_bin = df.groupby('discount_bin')['profit_margin'].mean().sort_values()
pm_by_bin

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,5))
# sns.barplot(
#     x=pm_by_bin.index,
#     y=pm_by_bin.values
# )
# plt.title("Average Profit Margin by Discount Range")
# plt.xlabel("Discount Range")
# plt.ylabel("Avg Profit Margin")
# plt.show()

# Interpretation:
# Low discounts (0–10%) deliver the highest average profit margin, making them the most financially healthy pricing range.
# Margins decline steadily as discounts increase, with 10–20% still positive but significantly lower.
# From 20–40% discount, average margins turn negative, showing that these sales generally lose money.
# The 40%+ discount range has extremely negative margins, confirming that deep discounting is highly unprofitable.
# Overall, the margin trend reinforces that higher discounts directly decrease profitability, with a clear break-even point around the 20% discount level.

# 5.6. Category-Level Discount Behavior
# Goal: identify categories that depend heavily on discounting.

category_discount = df.groupby('category')['discount'].mean().sort_values(ascending=False)
category_discount

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,5))
# sns.barplot(
#     x=category_discount.values,
#     y=category_discount.index
# )
# plt.title("Average Discount by Category")
# plt.xlabel("Average Discount")
# plt.ylabel("Category")
# plt.show()

# Interpretation:
# Furniture has the highest average discount, indicating more aggressive markdowns or frequent promotions in this category.
# Office Supplies receive moderate discounts, suggesting competitive pricing or regular small markdowns.
# Technology has the lowest average discount, implying stronger pricing power and less need for promotional reductions.
# Overall, discounting varies significantly by category, with Furniture being most discount-sensitive and Technology being the most price-stable.

# 5.7 Identify Order with High Discounts and Negative Profit

high_discounts_losses = df[(df['discount'] > 0.3) & (df['profit'] < 0)]
high_discounts_losses

# 1. Quantify the problem
num_orders = len(high_discounts_losses) #1140
total_loss = high_discounts_losses['profit'].sum() #-127737.555
avg_loss = high_discounts_losses['profit'].mean() #112.05

# 2. Analyze which categories cause the most damage

high_discounts_losses['category'].value_counts()
# Office Supplies    680
# Furniture          320
# Technology         140
high_discounts_losses.groupby('category')['profit'].sum().sort_values()
# Office Supplies   -47140.1376
# Furniture         -43782.4392
# Technology        -36814.9782

# 3. Analyze which products are driving the losses
high_discounts_losses.groupby('product_name')['profit'].sum().sort_values().head()
# Cubify CubeX 3D Printer Double Head Print          -9239.9692
# GBC DocuBind P400 Electric Binding System          -6859.3896
# Lexmark MX611dhe Monochrome Laser Printer          -5269.9690
# GBC Ibimaster 500 Manual ProClick Binding System   -5098.5660
# GBC DocuBind TL300 Electric Binding System         -4162.0336

# Interpretation:
# A total of 1,140 orders had discounts above 30% and resulted in negative profit, creating a combined loss of $127,737 (avg. loss $112 per order).
# Office Supplies (680 orders) and Furniture (320 orders) account for the majority of loss-making transactions, with total losses of –$47,140 and –$43,783, respectively.
# The worst-performing products include Cubify 3D Printers, GBC binding machines, and Lexmark printers, each generating several thousand dollars of losses when sold with high discounts.
# Overall, discounting varies significantly by category, with Furniture being most discount-sensitive and Technology being the most price-stable.



### Discount Impact Analysis

- Discounts have a strong negative effect on profitability, with a –0.86 correlation between discount and profit margin.
- Low discounts (0–10%) generate the highest profit and healthiest margins.
- Profitability drops sharply above 20%, and discounts over 30% consistently generate losses.
- Furniture receives the highest average discounts, followed by Office Supplies; Technology is discounted the least.
- High-discount loss analysis shows 1,140 orders with discounts over 30% resulted in $127K total losses, mostly in Office Supplies and Furniture.
- Several high-cost items (3D printers, binding machines, printers) are major loss drivers when heavily discounted.

Conclusion:
- Deep discounting is highly unprofitable. The business should limit discounts above 20–30% and review pricing strategies for high-cost, discount-sensitive products.

In [ ]:
# Geographic Analysis

# Compare sales and profit by regions
# Analyze profit margin by region
# Identify the best and the worst regions by profit
# Explore the relationship between category and region (heatmap)

# 6.1 Sales and Profit by Region

# Goal: to see which regions generate the highest sales and which are the most profitable

region_perf = (
    df.groupby('region').agg({
        'sales': 'sum',
        'profit': 'sum',
        'profit_margin': 'mean'
    })
    .sort_values(by='profit',ascending=False)
    .reset_index()
)

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,4))
# sns.barplot(
#     data=region_perf.sort_values('sales', ascending=False),
#     x='sales',
#     y='region',
#     label='Sales'
# )
# sns.barplot(
#     data=region_perf.sort_values('sales', ascending=False),
#     x='profit',
#     y='region',
#     label='Profit'
# )
# plt.title("Sales & Profit by Region")
# plt.xlabel("Total Sales & Profit")
# plt.ylabel("Region")

# plt.tight_layout()
# plt.show()

### Interpretation
# - West is the strongest region, leading in both total sales and total profit, with the highest profit margin (≈21.9%).
# - East performs well with high sales and solid profit, maintaining a healthy profit margin (~16.7%).
# - South generates lower sales but still remains profitable, with margins similar to East.
# - Central is the only region operating at a negative profit margin (–10.4%), indicating pricing, cost, or discounting issues specific to this region.
# - Conclusion: West and East are high-performing regions, while Central requires deeper review to understand and fix its loss-making behavior.

# 6.2 Profit Margin by Region

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(7,4))
# sns.barplot(
#     data=region_perf.sort_values('profit_margin', ascending=False),
#     x='profit_margin',
#     y='region'
# )
# plt.title("Average Profit Margin by Region")
# plt.xlabel("Average Profit Margin")
# plt.ylabel("Region")
# plt.show()

# Interpretation
# West has the highest profit margin, making it the most profitable region.
# East and South show solid, positive margins, indicating healthy performance.
# Central is the only region with a negative profit margin, signaling operational or pricing issues that require attention.

# 6.3
# Sales and Profit by States
# Goal: drill down to states to see where the profit comes and where it's lost

states_perf = (
    df.groupby('state').agg({
        'sales': 'sum',
        'profit': 'sum',
        'profit_margin': 'mean'
    })
    .sort_values(by='profit',ascending=False)
    .reset_index()
)
# states_perf.head(10)

top_states_perf = states_perf.head(10)

# Visualization: Top 10 States by Profit
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,6))
# sns.barplot(
#     data=top_states_perf,
#     x='profit',
#     y='state'
# )
# plt.title("Top 10 States by Profit")
# plt.xlabel("Total Profit")
# plt.ylabel("State")
# plt.show()

# Interpretation
# California and New York are by far the most profitable states, each contributing the largest share of total profit.
# Washington and Michigan also perform well, forming a strong second tier of profitable states.
# States like Virginia, Indiana, and Georgia deliver moderate but solid profit levels.
# Kentucky, Minnesota, and Delaware appear at the lower end but still contribute positively.
# Conclusion: Profitability is highly concentrated in a few key states, with California and New York being the primary drivers of regional financial performance.

bottom_states_perf = states_perf.sort_values('profit').head(10)

# Visualization: Bottom 10 States by Profit
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,6))
# sns.barplot(
#     data=bottom_states_perf,
#     x='profit',
#     y='state'
# )
# plt.title("Bottom 10 States by Profit")
# plt.xlabel("Total Profit")
# plt.ylabel("State")
# plt.show()

# Interpretation
# Texas is the largest loss-making state by a wide margin, followed by Ohio and Pennsylvania, indicating major profitability issues in these markets.
# Illinois and North Carolina also generate substantial losses, forming a second tier of underperforming states.
# States like Colorado, Tennessee, Arizona, Florida, and Oregon show smaller but still negative profit levels.
# The trend suggests that losses are concentrated in a handful of key states, especially Texas, which requires deeper investigation into pricing, discounting, product mix, or operational costs.
# Conclusion: Losses are heavily concentrated in specific states—most notably Texas—highlighting the need for targeted corrective strategies in these regions.

# 6.4 Region vs Ctegory Profit Heatmap

# Goal: check which combinations of Region & Category work best

region_category = (
    df.groupby(['region', 'category'])['profit']
      .sum()
      .reset_index()
)

pivot_rc = region_category.pivot(index="region", columns="category", values="profit")
pivot_rc

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(8,5))
# sns.heatmap(
#     pivot_rc,
#     annot=True,
#     fmt=".0f"
# )
# plt.title("Profit by Region and Category")
# plt.xlabel("Category")
# plt.ylabel("Region")
# plt.show()

# Interpretation
# West is the strongest region overall, driven by very high profits in Office Supplies and strong performance in Technology.
# East also performs well across all categories, with especially strong profit in Technology and Office Supplies.
# South shows moderate profitability in every category, with balanced results but lower totals than West and East.
# Central underperforms: Furniture is unprofitable, and although Technology and Office Supplies generate positive profit, they are much lower than in other regions.

# 6.5 Prepare Data for Mapping (for PowerBI)
# Goadl: we won't draw the actual geographical map in notebook, but will prepare a clean dataset

geo_export = states_perf.copy()
geo_export.to_csv("../output/state_performance.csv", index=False)
geo_export.head()


### Geographic Analysis

- West is the srongest region overall, leading in both sales and profit, with the srongest profit margins in all product categories
- East is close second one, consistently profitable with solid margins, especially in Technology and Office Suppliers
- South performs moderately well, generating positive profit in all produt categories, bu less than for West and East
- Central underperforms significantly, the only one showing overall negative profit margin, driven largerly by losses in Furniture Category
- At State level, California and New Yourk are the biggest contributers, while Texas, Ohio, Pennsylvania are the biggest loss-making states.

In [ ]:
# 7. Shipping & Operational Metrics

# Shipping time is an important operational KPI.
# Long delays can reduce customer satisfaction, increase return rates, and impact profit indirectly (especially for large or fragile items).

# Messure average shipping delay
# Compare average delay by ship mode
# See whether delay affects profit or correlates with discount
# Identify operational inefficiencies across the dataset

# 7.1 Explore Shipping Delay Distribution
# Goal: understand how many days it usually takes to deliver orders.

df['ship_delay'] = (df['ship_date'] - df['order_date']).dt.days
df['ship_delay'].describe()

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(8,5))
# sns.histplot(df['shipping_delay'], bins=20, kde=True)
# plt.title("Distribution of Shipping Delay (Days)")
# plt.xlabel("Days")
# plt.ylabel("Count of Orders")
# plt.show()

# Most orders are delivered in 2–6 days, with 4 days being the most common delivery time.
# A smaller number of orders arrive same-day (0 days) or after 7 days, indicating occasional extremes.
# The distribution shows a slight right-skew, meaning delays of 5–7 days occur more often than very fast deliveries.
# Overall, shipping performance is consistent, but long-delay cases (6–7 days) may need operational review.

# 7.2 Compare ship delays by Shipping Mode

shipmode_delay = (
    df.groupby('ship_mode')['shipping_delay']
      .mean()
      .sort_values()
)

shipmode_delay

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(8,5))
# sns.barplot(
#     x=shipmode_delay.values,
#     y=shipmode_delay.index
# )
# plt.title("Average Shipping Delay by Ship Mode")
# plt.xlabel("Avg Delay (Days)")
# plt.ylabel("Ship Mode")
# plt.show()

# Interpretation
# Same Day shipping performs as expected with near-zero delay,
# First Class has the lowest delay among regular shipping options (~2.2 days), showing strong reliability.
# Second Class averages a moderate delay (~3.2 days), aligned with its slower service level.
# Standard Class has the longest delay (~5 days), which is expected but still significantly longer than other modes.


# 7.3. Shipping delay by Category

shipmode_delay = (
    df.groupby('category')['shipping_delay']
      .mean()
      .sort_values()
)

shipmode_delay

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(8,5))
# sns.barplot(
#     x=shipmode_delay.values,
#     y=shipmode_delay.index
# )
# plt.title("Average Shipping Delay by Category")
# plt.xlabel("Avg Shipping Delay (Days)")
# plt.ylabel("Category")
# plt.show()

# Iterpretation
# All three categories (Furniture, Technology, Office Supplies) have very similar average shipping delays, around 4 days.
# This suggests delays are driven more by shipping processes than by product type.


# 7.4 Correlation between Shipping Delay & Profit
# Goal: see whether shipping delay impacts profitability.

df[['shipping_delay', 'profit', 'discount']].corr()

# Interpretation:
# Shipping delay has no meaningful correlation with profit (−0.004). Delivery speed does not affect profitability in this dataset.
# Discount has a moderate negative correlation with profit (−0.22), confirming that higher discounts reduce profit.
# Shipping delay and discount are uncorrelated (~0.0004), meaning discounting does not influence how fast orders are delivered.

# 7.5 Shipping delay vs Discount

# Goal: investigate if heavily discounted orders have shorted or longer shipping delays

# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(8,5))
# sns.scatterplot(
#     data=df,
#     x='shipping_delay',
#     y='discount',
#     alpha=0.3
# )
# plt.title("Shipping Delay vs Discount")
# plt.xlabel("Shipping Delay (Days)")
# plt.ylabel("Discount")
# plt.show()

# Interpretation:
# Order discounts evenly distributed across different shipping days, that means no clear relationship between shipping delay and discount

# 7.6. Flag Problematic Orders (Long Delay + Low Profit)

problematic_orders = df[(df['shipping_delay'] > 6) & (df['profit'] < 0)]
problematic_orders

# 1. Number of problematic orders
num_problem = len(problematic_orders) 

# 2. Total loss
total_loss = df['profit'].sum() 

# 3. Average loss per order
avg_loss_per_order = problematic_orders['profit'].mean()

# 4. Identify which categories are responsible
problematic_orders['category'].value_counts()
problematic_orders.groupby('category')['profit'].sum().sort_values()


# 5. Check product types
problematic_orders.groupby('product_name')['profit'].sum().sort_values().head(10)

# 6. Check regions
problematic_orders.groupby('region')['profit'].sum().sort_values()


# 7. Check ship mode
problematic_orders.groupby('ship_mode')['profit'].sum().sort_values()

# Interpretation

# 115 orders suffered from both long shipping delays (>6 days) and negative profit, creating operational and financial issues.
# These orders generated a total loss of –$16,491, with an average loss of –$143 per order.

# Which categories are responsible

# Most problematic orders come from Office Supplies (64 orders), followed by Furniture (35) and Technology (16).
# However, Technology orders caused the largest dollar loss (–$8.3K), indicating that delayed tech items are especially costly

# Worst products

# Losses are driven by bulky or high-shipping-cost items such as:
# Cubify 3D Printers,
# Binding machines,
# Large conference tables,
# Specialized office equipment.

# Regional impact

# Losses are concentrated in the East (–$9.2K), followed by South, West, and Central,

# Shipping mode

# 100% of problematic orders came from Standard Class.

# Key Insight
# Long delays combined with Standard Class shipping and heavy/bulky products—especially Technology—are the primary drivers of avoidable losses.


### Shipping Analysis Summary

- Most orders arrive within 2–6 days, with 4 days being the peak; delays of 6–7 days are operational outliers.
- Shipping speed aligns with expectations: Same Day is fastest, First Class performs well, and Standard Class is the slowest.
- All product categories (Furniture, Technology, Office Supplies) show similar average delays (~4 days), meaning delays come from logistics, not product type.
- Shipping delay has no meaningful correlation with profit — delivery time does not directly affect profitability.
- However, 115 problematic orders had long delays + negative profit, causing $16.5K in losses.
- These issues are driven by bulky, high-cost items, especially in Technology, and occur mostly in the East region.
- Standard Class accounts for all problematic orders

Conclusion:
Loss-making delayed orders are rare but predictable — they are heavy items shipped via Standard Class. Optimizing shipping strategy for these SKUs would significantly reduce operational losses.

In [ ]:
df.describe()

Key Business Insights
1. Overall Business Performance

The Superstore generated ≈$2.3M in total sales over the analyzed period.
Total profit ≈ $286K, resulting in an overall profit margin of ~12.5%.
There were ~5,000 total orders, showing moderate order volume for a retail operation of this scale.

2. Category-Level Profitability

Technology is the most profitable category, contributing the largest share of total profit.
Office Supplies is profitable but significantly less than Technology.
Furniture shows very low profitability and contains several loss-making sub-categories.

Insight: Business should prioritize Technology while reviewing discounting and pricing strategy for Furniture.

3. Sub-Category Performance

Phones and Chairs lead in sales volume.
Several sub-categories such as Tables, Bookcases, and high-ticket items in Furniture have consistently negative profit margins.

Insight: These unprofitable sub-categories may require renegotiation with suppliers, better pricing strategy, or reduced discounting.

4. Regional Performance

West and East regions show the strongest sales & profit performance.
South region performs moderately.
Central region contains the highest concentration of negative-profit items, pulling down overall profitability.

Insight: Central region requires deeper investigation into pricing, logistics costs, or customer mix.

5. Discount Impact

Scatterplot analysis shows:
Discounts above 20–30% often drive profit into negative territory.
Many high-discount transactions have loss-making profit margins.
Discounting is a major driver of profit erosion.

Insight: Implement stricter discount policies, especially for Furniture & Central region customers.

6. Customer Profitability

Profit contribution is heavily skewed — a classic Pareto (80/20) pattern.
A small group of top customers drives a significant portion of profit.
“Top 10 Customers by Profit” consistently deliver strong value.
Conversely, “Bottom 10 Products” and some customer segments contribute negative profit, mostly due to discount-heavy orders.

Insight: Strengthen relationships with top customers, and evaluate loss-making customer/product patterns.

7. Profit Margin Distribution

Margin distribution is highly skewed:
Most orders fall between 0% and 20% margin.
A long left-tail of orders exhibits negative margins, revealing opportunities for margin improvement.

Insight: Many products operate at razor-thin margins—pricing strategy requires optimization.

8. Product-Level Risk
Bottom 10 most unprofitable products show:
Losses up to -$8K per product.
Majority belong to Furniture and high-discount items.
These products represent repeated loss-making behavior.

Insight: These products require immediate intervention—either reprice, reduce discounting, or remove from catalog.

9. Seasonal Trends

Monthly sales trend shows clear seasonality, with:
Sales spikes occurring in late-year months.
Noticeable mid-year dips.

Insight: Demand forecasting and inventory planning should account for seasonal peaks.

Summary: Strategic Recommendations

Reduce aggressive discounting, especially above 20%.
Reassess unprofitable products (Tables, Bookcases, some high-ticket Furniture items).
Improve profitability in the Central region through logistics, pricing, or customer segmentation review.
Enhance pricing strategy for margin-sensitive categories.
Focus marketing efforts on top customers who consistently deliver profit.
Optimize the product portfolio, removing or repricing persistent loss-makers.